In [1]:
# !pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install transformers
!pip -q install accelerate>=0.12.0
!pip install datasets
!pip install evaluate
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.1 MB/s eta 0:00:00
  

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/SCIQA

Mounted at /content/drive
/content/drive/MyDrive/SCIQA


In [3]:
from datasets import load_dataset, Dataset, DatasetDict

def clean(st):
    st = st.replace("\n", " ")
    st = st.replace("?", " ?")
    st = st.replace("{", " { ")
    st = st.replace("}", " } ")
    st = st.replace("\\'", "'")

    while "  " in st:
        st = st.replace("  ", " ")
    return st.strip()


def get_entities(query):
    query = clean(query)
    entities = []
    relations = []
    all_good = []
    words = query.split(" ")
    for word in words:
        if word.startswith("orkg"):
            all_good.append(word)

    for word in all_good:
        if word.startswith("orkgp:"):
            relations.append(word)
        else:
            entities.append(word)

    return {"entities": entities, "relations": relations}


def get_dataset():
    new_dataset_dict = {}
    dataset = load_dataset("orkg/SciQA")
    # dataset2 = load_dataset("awalesushil/DBLP-QuAD")

    for split in dataset:

        question = []
        entities = []
        relations = []
        query = []

        for sample in dataset[split]:
            extracted_entities = get_entities(sample["query"]["sparql"])
            question.append(sample["question"]["string"])
            query.append(clean(sample["query"]["sparql"]))
            entities.append(extracted_entities["entities"])
            relations.append(extracted_entities["relations"])

        # for sample in dataset2[split]:
        #     question.append(sample["question"]["string"])
        #     query.append(clean(sample["query"]["sparql"]))
        #     entities.append(sample["entities"])
        #     relations.append(sample["relations"])

        new_dataset_dict[split] = {"question": question, "sparql_query": query, "entities": entities, "relations": relations}

    new_dataset_train = Dataset.from_dict(new_dataset_dict["train"])
    new_dataset_test = Dataset.from_dict(new_dataset_dict["test"])
    new_dataset_val = Dataset.from_dict(new_dataset_dict["validation"])
    new_dataset = DatasetDict({"train": new_dataset_train, "test": new_dataset_test, "validation": new_dataset_val})
    return new_dataset

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
import numpy as np

import os
os.environ["WANDB_DISABLED"] = "true"

checkpoint = "T5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
new_words = ['{', '}']
tokenizer.add_tokens(new_words)

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint,return_dict=True)
model.resize_token_embeddings(len(tokenizer))

prefix = "translate English to Sparql: "
metric = evaluate.load("sacrebleu")


def preprocess_function(examples):
    inputs = []
    targets = []
    for i, question in enumerate(examples["question"]):
        answer = examples["sparql_query"][i]
        entities = examples["entities"][i]
        relations = examples["relations"][i]
        inputs.append(prefix + question + "\nentities: " + str(entities) + "\nrelations: " + str(relations))
        targets.append(answer)
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


data = get_dataset()
print(data)
tokenized_books = data.map(preprocess_function, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="sciqa_T5_model_we_",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("sciqa_we_T5_model_we")
tokenizer.save_pretrained("sciqa_we_T5_model_we")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

Map:   0%|          | 0/1795 [00:00<?, ? examples/s]

Map:   0%|          | 0/513 [00:00<?, ? examples/s]

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-4-54af536f3946>:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,0.280023,0.003000,20.000000
2,No log,0.169623,0.004100,20.000000
3,1.007900,0.145480,0.004600,20.000000
4,1.007900,0.126375,0.004600,20.000000
5,0.149100,0.109097,0.004600,20.000000
6,0.149100,0.096180,0.004600,20.000000
7,0.112400,0.086946,0.004700,20.000000
8,0.112400,0.079812,0.004800,20.000000
9,0.085800,0.075709,0.004700,20.000000
10,0.085800,0.071158,0.004700,20.000000
